In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # loads GOOGLE_API_KEY from .env
assert os.getenv("GOOGLE_API_KEY"), "Missing GOOGLE_API_KEY in .env"

In [3]:
%pip install -q -U langchain langchain-chroma langchain-google-genai google-generativeai python-dotenv pypdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

In [6]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [7]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [8]:
vector_store = Chroma(
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    persist_directory='my_chroma_db_gemini',
    collection_name='sample_gemini'
)

In [9]:
# add documents
vector_store.add_documents(docs)

['4dab50c6-585b-4ace-a184-5abc7b4dd4bf',
 '4853c269-fbef-4c89-a81f-7ab1c6be62dd',
 '8357cacd-ab70-4772-8663-21e6a894d341',
 'b2da5271-3568-492e-bc4c-7784bd91fc55',
 '88059c3f-8d26-4a1e-8d2d-d32a89f0a2f1']

In [10]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4dab50c6-585b-4ace-a184-5abc7b4dd4bf',
  '4853c269-fbef-4c89-a81f-7ab1c6be62dd',
  '8357cacd-ab70-4772-8663-21e6a894d341',
  'b2da5271-3568-492e-bc4c-7784bd91fc55',
  '88059c3f-8d26-4a1e-8d2d-d32a89f0a2f1'],
 'embeddings': array([[ 1.35614481e-02, -7.62679800e-03, -2.51727290e-02, ...,
          1.35761239e-02,  3.51136659e-05, -7.35754846e-03],
        [ 3.18830460e-02, -3.36795598e-02, -4.39120680e-02, ...,
          1.08157219e-02, -1.05898809e-02, -1.52005616e-03],
        [ 1.58164389e-02,  3.38992523e-03, -2.23509353e-02, ...,
          1.95286470e-03, -7.18898326e-02, -8.88233818e-03],
        [ 1.01624681e-02, -5.16264960e-02, -2.89599448e-02, ...,
          3.29840817e-02, -2.09922232e-02,  1.78275332e-02],
        [-8.94617348e-04,  1.97164640e-02, -9.64438636e-03, ...,
          2.34126463e-03, -1.33365402e-02, -1.25717660e-02]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style 

In [11]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(id='b2da5271-3568-492e-bc4c-7784bd91fc55', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='4dab50c6-585b-4ace-a184-5abc7b4dd4bf', metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.')]

In [12]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(id='b2da5271-3568-492e-bc4c-7784bd91fc55', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.770696222782135),
 (Document(id='4dab50c6-585b-4ace-a184-5abc7b4dd4bf', metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.'),
  0.8030722737312317)]

In [13]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='88059c3f-8d26-4a1e-8d2d-d32a89f0a2f1', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.871087908744812),
 (Document(id='8357cacd-ab70-4772-8663-21e6a894d341', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.8778100609779358)]

In [14]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)


In [15]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4dab50c6-585b-4ace-a184-5abc7b4dd4bf',
  '4853c269-fbef-4c89-a81f-7ab1c6be62dd',
  '8357cacd-ab70-4772-8663-21e6a894d341',
  'b2da5271-3568-492e-bc4c-7784bd91fc55',
  '88059c3f-8d26-4a1e-8d2d-d32a89f0a2f1'],
 'embeddings': array([[ 1.35614481e-02, -7.62679800e-03, -2.51727290e-02, ...,
          1.35761239e-02,  3.51136659e-05, -7.35754846e-03],
        [ 3.18830460e-02, -3.36795598e-02, -4.39120680e-02, ...,
          1.08157219e-02, -1.05898809e-02, -1.52005616e-03],
        [ 1.58164389e-02,  3.38992523e-03, -2.23509353e-02, ...,
          1.95286470e-03, -7.18898326e-02, -8.88233818e-03],
        [ 1.01624681e-02, -5.16264960e-02, -2.89599448e-02, ...,
          3.29840817e-02, -2.09922232e-02,  1.78275332e-02],
        [-8.94617348e-04,  1.97164640e-02, -9.64438636e-03, ...,
          2.34126463e-03, -1.33365402e-02, -1.25717660e-02]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style 

In [16]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [17]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4dab50c6-585b-4ace-a184-5abc7b4dd4bf',
  '4853c269-fbef-4c89-a81f-7ab1c6be62dd',
  '8357cacd-ab70-4772-8663-21e6a894d341',
  'b2da5271-3568-492e-bc4c-7784bd91fc55',
  '88059c3f-8d26-4a1e-8d2d-d32a89f0a2f1'],
 'embeddings': array([[ 1.35614481e-02, -7.62679800e-03, -2.51727290e-02, ...,
          1.35761239e-02,  3.51136659e-05, -7.35754846e-03],
        [ 3.18830460e-02, -3.36795598e-02, -4.39120680e-02, ...,
          1.08157219e-02, -1.05898809e-02, -1.52005616e-03],
        [ 1.58164389e-02,  3.38992523e-03, -2.23509353e-02, ...,
          1.95286470e-03, -7.18898326e-02, -8.88233818e-03],
        [ 1.01624681e-02, -5.16264960e-02, -2.89599448e-02, ...,
          3.29840817e-02, -2.09922232e-02,  1.78275332e-02],
        [-8.94617348e-04,  1.97164640e-02, -9.64438636e-03, ...,
          2.34126463e-03, -1.33365402e-02, -1.25717660e-02]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style 